In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available())

False


In [56]:
# Vamos a utilizar el dataset de MNIST pero en vez de utilizar las redes lineas normales
# vamos a utilizar CNN
# Utilizamos redes convolucionales para obtener patrones de imagenes, para posteriormente realizar
# la clasificación de las imágenes necesitamos utilziar redes totalmente conectadas (nn.Linear)

class CNN(nn.Module):

    # __init__ es el constructor en Python, nos permitirá definir las capas que conformaran nuestro modelo pero no 
    # se encuentran ya estructuradas, sino sólo definidas
    def __init__(self, num_entradas, num_clases):

        super(CNN, self).__init__()
        # Es una imagen en blanco en negro por lo que es una matriz con su ancho y largo pero no tiene colores
        self.conv1 = nn.Conv2d(in_channels = num_entradas, out_channels = 32, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        self.pool = nn.MaxPool2d(kernel_size = (2, 2), stride = (2, 2))
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
        # 7 x 7 x 64
        self.fc1 = nn.Linear(in_features = 3136, out_features = 128)
        self.fc2 = nn.Linear(in_features = 128, out_features = num_clases)

        # La salida no se define

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(input = x, p = 0.5)
        x = self.fc2(x)

        return x

In [57]:
# Hiperparámetros
num_entradas = 1
num_clases = 10
batch_size = 64
learning_rate = 1e-3
num_epocas = 3

In [58]:
# Vamos a descargar el dataset de MNIST
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = datasets.MNIST(root = 'dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [59]:
# Inicializamos el modelo
# Usamos .to(device) para llevar el entrenamiento del modelo al dispositivo disponible (GPU o CPU)
modelo = CNN(num_entradas = num_entradas, num_clases = num_clases).to(device)

In [60]:
# Loss function, para clasificaciones de una sola etiqueta (y) como en este caso podemos utilizar Cross Entropy Loss
func_perdida = nn.CrossEntropyLoss()
optimizador = optim.Adam(params = modelo.parameters(), lr = learning_rate)

In [61]:
cont = 0

for epoca in range(num_epocas):

    for batch_idx, (x, y) in enumerate(iterable = train_loader):

        x = x.to(device = device)
        y = y.to(device = device)

        if (cont == 0):
            
            print(x.shape)
            cont += 1

        # forward
        scores = modelo(x)
        loss = func_perdida(scores, y)

        # Backpropagation
        # Zero your gradients for every batch!
        optimizador.zero_grad() # Ponemos el gradiente a 0 para cada batch para que no almacene las derivadas de las anteriores capas
        loss.backward()

        # Descenso del gradiente o pasos del algoritmo ADAM
        optimizador.step()

torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 28, 28])
torch.Size([64, 1, 2

In [64]:
# Comprobar precisión del modelo en entrenamiento y validacion
def comprobar_precision(loader, model):

    num_correct = 0
    num_samples = 0
    model.eval()

    # Para evaluar el modelo no necesitamos calcular el gradiente
    with torch.no_grad(): 
    
        for x, y in loader:
    
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Tenemos {num_correct} / {num_samples} con una precision de {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()

In [65]:
comprobar_precision(train_loader, modelo)
comprobar_precision(test_loader, modelo)

Tenemos 58899 / 60000 con una precision de 98.17
Tenemos 9790 / 10000 con una precision de 97.90
